In [ ]:
import argparse
import os
os.environ['SPARK_VERSION'] = '3.1'
os.environ['JAVA_HOME'] = '/Library/Java/JavaVirtualMachines/jdk-11.0.12.jdk/Contents/Home'

In [ ]:
import pydeequ
from pydeequ.analyzers import *
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession\
    .builder\
    .config("spark.jars.packages", pydeequ.deequ_maven_coord)\
    .config("spark.jars.excludes", pydeequ.f2j_maven_coord)\
    .getOrCreate()

In [ ]:
df = spark.read.parquet('./data/data.parquet')

In [ ]:
df.printSchema()

## PyDeeque Analyzer Research

In [ ]:
analyzer = AnalysisRunner(spark)\
    .onData(df)\
    .addAnalyzer(Size())\
    .addAnalyzer(Completeness('name'))\
    .run()

In [ ]:
success_metrics_df = AnalyzerContext.successMetricsAsDataFrame(spark, analyzer)

In [ ]:
success_metrics_df.show()